### Import des données

In [218]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [219]:
df_corkstation = pd.read_csv("Corkstation_15072019_01022011.csv")
df_corkstation.head()

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,2009-07-15 00:00:00,3,0.0,0,11.6,0,11.4,11.3,13.3,98,...,2,6,2,310,2,81,0.0,25000,999,1
1,2009-07-15 01:00:00,3,0.0,0,11.4,0,11.1,10.8,13.0,96,...,2,6,2,310,2,11,0.0,30000,999,1
2,2009-07-15 02:00:00,3,0.1,0,11.2,0,10.9,10.6,12.8,96,...,2,7,2,290,2,11,0.0,30000,999,1
3,2009-07-15 03:00:00,3,0.0,0,11.1,0,10.8,10.5,12.7,96,...,2,8,2,300,2,11,0.0,30000,999,1
4,2009-07-15 04:00:00,3,0.0,0,11.4,0,11.1,10.9,13.0,96,...,2,7,2,310,2,11,0.0,30000,999,1


In [220]:
df_vst = pd.read_csv("df_vst_group6.csv")
df_vst.head()

,time,mean,temp
0,2009-07-15 00:00:00,0.341659,11.6
1,2009-07-15 00:30:00,0.274463,11.5
2,2009-07-15 01:00:00,0.231365,11.4
3,2009-07-15 01:30:00,0.202708,11.3
4,2009-07-15 02:00:00,0.182256,11.2


### Prétraitement


Excluding incomplete data from df_corkstation

In [221]:
missing_indexes = np.where(df_corkstation.map(lambda x: x == ' '))[0]
df_corkstation = df_corkstation.drop(index=missing_indexes)

Resample and interpolating

In [222]:
# Changing index to date column
df_corkstation['date'] = pd.to_datetime(df_corkstation['date'])
df_corkstation.set_index('date', inplace=True)

# Changing to numeric form
df_corkstation['vis'] = pd.to_numeric(df_corkstation['vis'])
df_corkstation['wetb'] = pd.to_numeric(df_corkstation['wetb'])

# Resample for each 30 min and interpolating
df_corkstation_resampled = df_corkstation.resample('30T').interpolate(method='linear')

df_corkstation_resampled.head()


,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
date,,,,,,,,,,,,,,,,,,,,
2009-07-15 00:00:00,3.0,0.00,0.0,11.6,0.0,11.40,11.30,13.30,98.0,1002.80,2.0,6.0,2.0,310.0,2.0,81.0,0.0,25000.0,999.0,1.0
2009-07-15 00:30:00,3.0,0.00,0.0,11.5,0.0,11.25,11.05,13.15,97.0,1003.15,2.0,6.0,2.0,310.0,2.0,46.0,0.0,27500.0,999.0,1.0
2009-07-15 01:00:00,3.0,0.00,0.0,11.4,0.0,11.10,10.80,13.00,96.0,1003.50,2.0,6.0,2.0,310.0,2.0,11.0,0.0,30000.0,999.0,1.0
2009-07-15 01:30:00,3.0,0.05,0.0,11.3,0.0,11.00,10.70,12.90,96.0,1003.95,2.0,6.5,2.0,300.0,2.0,11.0,0.0,30000.0,999.0,1.0
2009-07-15 02:00:00,3.0,0.10,0.0,11.2,0.0,10.90,10.60,12.80,96.0,1004.40,2.0,7.0,2.0,290.0,2.0,11.0,0.0,30000.0,999.0,1.0


### Analyse exploratoire

### Modèle

### Validation

MAE, RMSE, MAPE ...